In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars import selectors as cs
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
# from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from typing_extensions import Optional
from data_display.input_data_plots import plot_basin_height_volume_table
from baseline_model.optimization_results_processing import *
from data_display.baseline_plots import *
from utility.pyomo_preprocessing import *
from utility.input_data_preprocessing import *
from config import settings
from general_function import pl_to_dict, pl_to_dict_with_tuple, build_non_existing_dirs, generate_log, duckdb_to_dict

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.graph_objects import Figure

from plotly.subplots import make_subplots

import networkx as nx
from data_display.baseline_plots import *
from baseline_model.baseline_input import BaseLineInput
from baseline_model.first_stage.first_stage_pipeline import BaselineFirstStage
from baseline_model.second_stage.second_stage_pipeline import BaselineSecondStage
COLORS = px.colors.qualitative.Plotly

log = generate_log(name="test")

os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"
volume_factor = 1e-6

In [2]:
input_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
# smallflex_input_schema: SmallflexInputSchema = SmallflexInputSchema().duckdb_to_schema(file_path=input_file_names["duckdb_input"])


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.FILE_NAMES)) # type: ignore
PARALLEL = False
YEARS = [2020, 2021, 2022, 2023]
TURBINE_FACTORS = {0.75, 0.85, 0.95}

SIMULATION_SETTING = [
    {"quantile": 0, "buffer": 0.1, "powered_volume_enabled": True},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False},
    # {"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True, "global_price": True},
    # {"quantile": 0.25, "buffer": 0.3, "powered_volume_enabled": False, "global_price": True},
]

YEARS = 2020
TURBINE_FACTORS = 0.75
# SIMULATION_SETTING = [{"quantile": 0.15, "buffer": 0.3, "powered_volume_enabled": True}]

REAL_TIMESTEP = timedelta(hours=1)
FIRST_STAGE_TIMESTEP = timedelta(days=1)
SECOND_STAGE_TIME_SIM = timedelta(days=4)
TIME_LIMIT = 20 # in seconds
VOLUME_FACTOR = 1e-6

baseline_input: BaseLineInput = BaseLineInput(
    input_schema_file_name=output_file_names["duckdb_input"],
    real_timestep=REAL_TIMESTEP,
    year=YEARS,
    max_alpha_error=2,
    hydro_power_mask = c("name").is_in(["Aegina discrete turbine", "Aegina pump"]),
    volume_factor=VOLUME_FACTOR
)
first_stage: BaselineFirstStage = BaselineFirstStage(
    nb_state= 1,
    input_instance=baseline_input,
    timestep=FIRST_STAGE_TIMESTEP,
    max_turbined_volume_factor=TURBINE_FACTORS
)
first_stage.solve_model()


second_stage: BaselineSecondStage = BaselineSecondStage(
        input_instance=baseline_input, 
        first_stage=first_stage, 
        timestep=SECOND_STAGE_TIME_SIM, 
        time_limit=TIME_LIMIT,
        model_nb=0,
        nb_state=2,
        is_parallel=PARALLEL,
        **SIMULATION_SETTING[0]
    )
second_stage.solve_model()

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 15/15 [00:01<00:00,  8.50it/s]
Solving first stage optimization problem: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.67it/s]
Solving second stage optimization model number 0:   9%|█████▌                                                          | 8/92 [00:37<03:46,  2.69s/it]

2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Read LP format model from file /tmp/tmpk1ubdfpm.pyomo.lp
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Reading time = 0.00 seconds
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO x1: 2508 rows, 1638 columns, 6350 nonzeros
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Set parameter TimeLimit to value 20
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Set parameter Threads to value 8
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.5 LTS")
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO 
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO Thread count: 2 physical cores, 4 logical processors, using up to 8 threads
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INFO 
2025-06-16 17:33:05 B01BPC15 gurobipy[196086] INF

2025-06-16 17:33:05 B01BPC15 baseline_model.second_stage.second_stage_pipeline[196086] ERROR Model not solved for sim number 8
Solving second stage optimization model number 0:   9%|█████▌                                                          | 8/92 [00:37<06:32,  4.68s/it]


In [4]:
optimization_summary, combined_results = combine_second_stage_results(
    optimization_results= second_stage.optimization_results,
    powered_volume= second_stage.powered_volume,
    market_price=second_stage.market_price, 
    index=second_stage.index, 
    flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor)

KeyError: 'power'

In [ ]:
list(second_stage.model_instance.HS)    

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9)]

In [ ]:
index_list: list[str] = list(map(lambda x: x.name, getattr(second_stage.model_instance, "flow_by_state").index_set().subsets()))

In [ ]:
getattr(second_stage.model_instance, "flow_by_state")

In [ ]:
index_list



['T', 'HS']

In [ ]:
getattr(second_stage.model_instance, "max_flow").extract_values().items()

dict_items([((0, 0), 2.4686280000000003), ((0, 1), 2.4728100000000004), ((0, 2), 2.4767435000000004), ((0, 3), 2.480665), ((0, 4), 2.484585), ((1, 5), 2.3843599999999996), ((1, 6), 2.37365), ((1, 7), 2.36331), ((1, 8), 2.351722), ((1, 9), 2.339978)])

In [ ]:
extract_optimization_results(model_instance=second_stage.model_instance, var_name="max_flow")

InvalidOperationError: cannot cast List type (inner: 'Int64', to: 'UInt32')

In [ ]:
list(second_stage.model_instance.S_BH)

[]

In [ ]:
getattr(second_stage.model_instance, "max_inactive_flow_by_state_constraint").pprint()

max_inactive_flow_by_state_constraint : Size=0, Index=T*S_BH, Active=True
    Key : Lower : Body : Upper : Active


In [ ]:
print(pl.DataFrame(
            map(list, getattr(second_stage.model_instance, "flow_by_state").extract_values().items()), 
            schema= ["index", "flow_by_state"]
).with_columns(
    c("index").list.to_struct(fields=["T", "H", "S_H"])
).unnest("index").pivot(on="S_H", values="flow_by_state", index="T").to_pandas().to_string())

     T    0        1         2    3    4    5    6        7    8    9
0    1  0.0  0.00000 -0.000000  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
1    2  0.0  0.00000  0.000000  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
2    3  0.0  0.00000  0.000000  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
3    4  0.0  0.00000 -0.000000  0.0  0.0  0.0  0.0  2.36331  0.0  0.0
4    5  0.0  0.00000  0.000000  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
5    6  0.0  0.00000  0.000000  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
6    7  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
7    8  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
8    9  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
9   10  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
10  11  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
11  12  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
12  13  0.0  0.00000  2.476744  0.0  0.0  0.0  0.0  0.00000  0.0  0.0
13  14  0.0  0.00000

In [ ]:
extract_optimization_results(model_instance=second_stage.model_instance, var_name="flow_by_state")

T,HS,flow_by_state
u32,u32,f64
1,0,0.0
1,0,0.0
1,0,0.0
1,0,0.0
1,0,0.0
…,…,…
96,1,0.0
96,1,0.0
96,1,0.0


In [ ]:
print(extract_optimization_results(
        model_instance=second_stage.model_instance, var_name="flow"
    ).pivot(on="H", values="flow", index="T").to_pandas().to_string())

     T           0      1
0    1    0.000000   0.00
1    2    0.000000   0.00
2    3    0.000000   0.00
3    4    0.000000   0.00
4    5    0.000000   0.00
5    6    0.000000   0.00
6    7    0.000000   0.00
7    8    0.000000   0.00
8    9    0.000000   0.00
9   10    0.000000   0.00
10  11    0.000000   0.00
11  12    0.000000   0.00
12  13    0.000000   0.00
13  14    0.000000   0.00
14  15    0.000000   0.00
15  16    0.000000   0.00
16  17    0.000000   0.00
17  18    0.000000   0.00
18  19    0.000000   0.00
19  20    0.000000   0.00
20  21    0.000000   0.00
21  22    0.000000   0.00
22  23    0.000000   0.00
23  24    0.000000   0.00
24  25    0.000000   0.00
25  26    0.000000   0.00
26  27    0.000000   0.00
27  28    0.000000   0.00
28  29    0.000000  26.88
29  30    0.000000   0.00
30  31    0.000000   0.00
31  32    0.000000   0.00
32  33    0.000000   0.00
33  34    0.000000   0.00
34  35    0.000000   0.00
35  36    0.000000   0.00
36  37    0.000000   0.00
37  38    0.

In [5]:
optimization_results= second_stage.optimization_results
powered_volume= second_stage.powered_volume
market_price=second_stage.market_price
index=second_stage.index
flow_to_vol_factor= second_stage.real_timestep.total_seconds() * second_stage.volume_factor

remaining_volume = pivot_result_table(
    df = optimization_results["remaining_volume"], on="H", index="sim_nb", 
    values="remaining_volume")

powered_volume = pivot_result_table(
    df = powered_volume, on="H", index="sim_nb", 
    values="powered_volume"
    ).with_columns(
        c("sim_nb").cast(pl.Int32).alias("sim_nb")
    )

real_powered_volume = pivot_result_table(
    df = optimization_results["flow"]
        .group_by("sim_nb", "H")
        .agg((c("flow").sum() * flow_to_vol_factor ).alias("real_powered_volume")),
    on="H", index="sim_nb", 
    values="real_powered_volume")

        
start_basin_volume = pivot_result_table(
    df = optimization_results["start_basin_volume"],
    on="B", index="sim_nb", 
    values="start_basin_volume")

optimization_summary = remaining_volume\
.join(powered_volume, on = "sim_nb", how="inner")\
.join(real_powered_volume, on = "sim_nb", how="inner")\
.join(start_basin_volume, on = "sim_nb", how="inner")

volume = optimization_results["flow"].with_columns((c("flow") * flow_to_vol_factor).alias("volume"))
volume = pivot_result_table(
    df = volume, on="H", index=["T", "sim_nb"], 
    values="volume", reindex=True)

power = pivot_result_table(
    df = optimization_results["hydro_power"], on="H", index=["T", "sim_nb"], 
    values="hydro_power", reindex=True)

volume_max_mapping: dict[str, float] = pl_to_dict(index["water_basin"][["B", "volume_max"]])
basin_volume = optimization_results["basin_volume"].with_columns(
    (c("basin_volume") / c("B").replace_strict(volume_max_mapping, default=None)).alias("basin_volume")
)

basin_volume = pivot_result_table(
    df = basin_volume, on="B", index=["T", "sim_nb"], 
    values="basin_volume", reindex=True)

spilled_volume = pivot_result_table(
    df = optimization_results["spilled_volume"], on="B", index=["T", "sim_nb"], 
    values="spilled_volume", reindex=True)

market_price = market_price.with_row_index(name="real_index")\
    .select(c("real_index"), c("avg").alias("market_price"))

combined_results = index["datetime"]\
    .with_row_index(name="real_index")[["real_index", "timestamp"]]\
    .join(basin_volume, on = "real_index", how="inner")\
    .join(volume, on = "real_index", how="inner")\
    .join(power, on = "real_index", how="inner")\
    .join(spilled_volume, on = "real_index", how="inner")\
    .join(market_price, on = "real_index", how="inner")\
    # .with_columns(
    #     (pl.sum_horizontal(cs.starts_with("power")) * c("market_price")).alias("income"),
    # )


In [7]:
remaining_volume


sim_nb,remaining_volume_0,remaining_volume_1
i32,f64,f64
0,0.0,0.0
1,-0.039967,-0.042278
2,0.007891,-0.11073
3,0.003076,-0.144757
4,-0.000264,-0.162052
5,-0.002201,-0.170744
6,0.085572,-0.179482
7,0.05981,-0.179482
8,0.105922,-0.179482


In [6]:
print(combined_results.to_pandas().to_string())

     real_index                 timestamp  basin_volume_0  basin_volume_1  volume_0  volume_1  hydro_power_0  hydro_power_1  spilled_volume_0  spilled_volume_1  market_price
0             0 2020-01-01 00:00:00+00:00        0.811261        0.000000  0.008934 -0.000000       8.467411       0.000000      0.000000e+00          0.000000         35.42
1             1 2020-01-01 01:00:00+00:00        0.810804        0.008934  0.008934  0.000000       8.467411       0.000000      0.000000e+00          0.000000         34.04
2             2 2020-01-01 02:00:00+00:00        0.810347        0.017868  0.000000  0.000000       0.000000       0.000000      0.000000e+00          0.000000         31.52
3             3 2020-01-01 03:00:00+00:00        0.810380        0.017868  0.000000  0.000000       0.000000       0.000000      0.000000e+00          0.000000         28.29
4             4 2020-01-01 04:00:00+00:00        0.810413        0.017868  0.000000  0.008456       0.000000     -11.294057      0

In [ ]:
list(find_infeasible_constraints(second_stage.model_instance))

[(<pyomo.core.base.constraint.ConstraintData at 0x7a626788ee40>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788eee0>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788ef30>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788ef80>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788efd0>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f020>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f070>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f0c0>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f110>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f160>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f1b0>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f200>, None, 4),
 (<pyomo.core.base.constraint.ConstraintData at 0x7a626788f250>, None, 4),
 (<pyomo.core.base.constr

In [ ]:
model_instance = second_stage.model_instance
optimization_results = second_stage.optimization_results
sim_nb = second_stage.sim_nb

for var_name in ["flow", "hydro_power", "basin_volume", "spilled_volume"]:
    data = extract_optimization_results(
            model_instance=model_instance, var_name=var_name
        ).with_columns(
            pl.lit(sim_nb).alias("sim_nb")
        )
    optimization_results[var_name] = pl.concat([optimization_results[var_name], data], how="diagonal_relaxed")
            
        
start_basin_volume = extract_optimization_results(
        model_instance=model_instance, var_name="end_basin_volume"
    ).with_columns(
        pl.lit(sim_nb + 1).alias("sim_nb")
    ).rename({"end_basin_volume": "start_basin_volume"})

remaining_volume = join_pyomo_variables(
        model_instance=model_instance, 
        var_list=["diff_volume_pos", "diff_volume_neg"], 
        index_list=["H"]
    ).select(
        c("H"),
        pl.lit(sim_nb + 1).alias("sim_nb"),
        (c("diff_volume_pos") - c("diff_volume_neg")).alias("remaining_volume"),
    )
optimization_results["start_basin_volume"] = pl.concat([optimization_results["start_basin_volume"], start_basin_volume], how="diagonal_relaxed")
optimization_results["remaining_volume"] = pl.concat([optimization_results["remaining_volume"], remaining_volume], how="diagonal_relaxed")

In [ ]:
optimization_results["remaining_volume"].tail(10)

H,sim_nb,remaining_volume
u32,i32,f64
0,16,-0.878202
1,16,-0.967658
0,17,-0.888381
1,17,-0.967669
0,18,-0.893471
1,18,-0.967674
0,19,-0.896015
1,19,-0.967677
0,20,null


In [ ]:
start_volume_dict = pl_to_dict(
self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])

discharge_volume_tot= pl_to_dict(
self.discharge_volume.filter(c("sim_nb") == self.sim_nb).group_by("B").agg(c("discharge_volume").sum()))

self.index["basin_state"], basin_volume = generate_seconde_stage_basin_state(
    index=self.index, water_flow_factor=self.water_flow_factor, 
    basin_volume_table=self.basin_volume_table, start_volume_dict=start_volume_dict, 
    discharge_volume_tot=discharge_volume_tot,
    timestep=self.timestep, volume_factor=self.volume_factor, nb_state=5
)

self.index["hydro_power_state"] = generate_second_stage_hydro_power_state(
    power_performance_table=self.power_performance_table, basin_volume=basin_volume)

NameError: name 'self' is not defined

In [ ]:
self.index["basin_state"]

S_b,volume_min,volume_max,B,BS
i64,f64,f64,i64,list[i64]
0,13.4132,13.88432,0,"[0, 0]"
1,13.94321,14.35982,0,"[0, 1]"
2,14.41944,14.83966,0,"[0, 2]"
3,14.9,15.32336,0,"[0, 3]"
4,15.38419,15.81,0,"[0, 4]"
5,0.0,1.0,1,"[1, 5]"


In [ ]:
performance_table["power_performance"]

NameError: name 'performance_table' is not defined

In [ ]:
new_hydro_state.sort("height").interpolate().with_columns(
    (c("power")/ c("flow")).alias("alpha")
)

head,flow,power,volume,alpha,height,S_b
f64,f64,f64,f64,f64,f64,f64
408.0,-2.3894,-11.3776,13.4132,4.761697,2378.0,0.0
408.0,-2.3894,-11.3776,13.4132,4.761697,2378.0,0.0
408.052632,-2.388737,-11.376189,13.444195,4.762429,2378.1,0.0
408.105263,-2.388074,-11.374779,13.475189,4.763161,2378.1,0.0
408.157895,-2.387411,-11.373368,13.506184,4.763893,2378.2,0.0
…,…,…,…,…,…,…
411.842105,-2.337158,-11.271316,15.713953,4.822659,2381.8,4.0
411.894737,-2.336385,-11.269811,15.745968,4.82361,2381.9,4.0
411.947368,-2.335613,-11.268305,15.777984,4.824561,2381.9,4.0


In [ ]:
start_volume_dict = self.optimization_results["start_volume"].to_dicts()[-1]

KeyError: 'start_volume'

In [ ]:
index_b = self.index["water_basin"]["B"].to_list()[0]
timestep = self.timestep

B,volume_max,volume_min,S_b
u32,f64,f64,i32
0,18.2432,0.056996,0


In [ ]:
basin_state

S_b,volume_min,volume_max,B
i64,f64,f64,i64
0,13.4132,13.88432,0
1,13.94321,14.35982,0
2,14.41944,14.83966,0
3,14.9,15.32336,0
4,15.38419,15.81,0
5,0.0,1.0,1


In [ ]:
basin_volume

S_b,height,B
u32,f64,i32
0,2378.0,0
0,2378.1,0
0,2378.2,0
0,2378.3,0
0,2378.4,0
…,…,…
4,2381.6,0
4,2381.7,0
4,2381.8,0


In [ ]:
arange_float(basin_volume["height"].max(), basin_volume["height"].min(), 0.1)

S_b,height,volume
u32,f64,f64
0,2378.0,13.4132
0,2378.1,13.47209
0,2378.2,13.53098
0,2378.3,13.58987
0,2378.4,13.64876
…,…,…
4,2381.6,15.56668
4,2381.7,15.62751
4,2381.8,15.68834


In [ ]:
water_flow

B,water_flow_in,water_flow_out,start_volume,discharge_volume,max_volume,min_volume,boundaries
i64,f64,f64,f64,f64,f64,f64,list[f64]
0,0.96768,-0.89856,14.8,0.028002,15.795682,13.90144,"[13.90144, 15.795682]"
1,0.89856,-0.96768,0.0,0.0,0.89856,-0.96768,"[-0.96768, 0.89856]"


In [ ]:
performance_table = self.power_performance_table[0]

start_volume = start_volume_dict[performance_table["B"]]

In [ ]:
height_boundary = data.select(pl.concat_list(c("height").min(), c("height").max()))["height"].to_list()[0]

In [ ]:
boundaries = 

In [ ]:
self = second_stage

start_volume_dict = pl_to_dict(
            self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])

discharge_volume_tot= pl_to_dict(
            self.discharge_volume.filter(c("sim_nb") == self.sim_nb).group_by("B").agg(c("discharge_volume").sum()))

In [ ]:
rated_flow_dict = pl_to_dict(self.index["hydro_power_plant"][["H", "rated_flow"]])


In [ ]:
rated_flow_dict

{0: 2.8, 1: None}

In [ ]:
state_index: pl.DataFrame = pl.DataFrame()
start_state: int = 0
for performance_table in power_performance_table:
    
    start_volume = start_volume_dict[performance_table["B"]]
    rated_volume = rated_flow_dict[performance_table["H"]] * timestep.total_seconds() * volume_factor
    
    boundaries = (
        start_volume - rated_volume, start_volume + rated_volume + discharge_volume[performance_table["B"]]
    )
    data: pl.DataFrame = filter_data_with_next(
        data=performance_table["power_performance"], col="volume", boundaries=boundaries)
    y_cols = data.select(cs.starts_with(name) for name in ["flow", "electrical"]).columns
    state_name_list = list(set(map(lambda x : x.split("_")[-1], y_cols)))
    
    data = define_state(data=data, x_col="volume", y_cols=y_cols, error_threshold=error_threshold)
    
    data = data\
        .with_row_index(offset=start_state, name="S")\
        .with_columns(
                pl.lit(performance_table["H"]).alias("H"),
                pl.lit(performance_table["B"]).alias("B")
        ).with_columns(
            pl.struct(cs.ends_with(col_name)).name.map_fields(lambda x: "_".join(x.split("_")[:-1])).alias(col_name)
            for col_name in state_name_list
        ).unpivot(
            on=state_name_list, index= ["volume", "S", "H", "B"], value_name="data", variable_name="state"
        ).unnest("data").drop("state")
        
    state_index = pl.concat([state_index, data], how="diagonal")
    start_state = state_index["S"].max() + 1 # type: ignore
    
state_index = state_index.with_row_index(name="S_Q")    
missing_basin: pl.DataFrame = index["water_basin"]\
    .filter(~c("B").is_in(state_index["B"]))\
    .select(
        c("B"),
        pl.struct(
            c("volume_min").fill_null(0.0).alias("min"), 
            c("volume_max").fill_null(0.0).alias("max"),
        ).alias("volume")
    ).with_row_index(offset=start_state, name="S")

index["state"] = pl.concat([state_index, missing_basin], how="diagonal_relaxed")\
    .with_columns(
    pl.concat_list("H", "B", "S", "S").alias("S_BH"),
    pl.concat_list("B", "S").alias("BS"),
    pl.concat_list("H", "S").alias("HS"),
    pl.concat_list("H", "S", "S_Q").alias("HQS")
)

NameError: name 'power_performance_table' is not defined

In [ ]:
discharge_volume_tot

{0: 0.028002449999999998}

In [ ]:
start_volume_dict

{0: 14.799999999999999, 1: 0.0}

In [ ]:


self.data["T"] = {None: self.index["datetime"].filter(c("sim_nb") == self.sim_nb)["T"].to_list()}
self.data["S_B"] = pl_to_dict(
    self.index["state"].unique("S", keep="first")
    .group_by("B", maintain_order=True).agg("S")
    .with_columns(c("S").list.sort())
)
self.data["BS"] = {None: list(map(tuple,self.index["state"]["BS"].to_list()))}


KeyError: 'state'

In [ ]:

self.data["HS"] = {None: list(map(tuple,hydropower_state["HS"].to_list()))}
self.data["HQS"] = {None: list(map(tuple,hydropower_state["HQS"].to_list()))}
self.data["S_H"] = pl_to_dict(
    hydropower_state.unique("S", keep="first")
    .group_by("H", maintain_order=True)
    .agg("S")
    .with_columns(c("S").list.sort())
    )
self.data["S_Q"] = pl_to_dict_with_tuple(
    hydropower_state
    .group_by(["HS"], maintain_order=True).agg("S_Q")
    .with_columns(c("S_Q").list.sort())
    )

self.data["B_H"] = pl_to_dict(hydropower_state.group_by("H").agg(c("B").unique()))
self.data["SB_H"] = pl_to_dict_with_tuple(hydropower_state.group_by("HS").agg(c("S").unique()))

self.data["start_basin_volume"] = pl_to_dict(
    self.optimization_results["start_basin_volume"].filter(c("sim_nb") == self.sim_nb)[["B", "start_basin_volume"]])
self.data["remaining_volume"] = pl_to_dict(self.optimization_results["remaining_volume"].filter(c("sim_nb") == self.sim_nb)[["H", "remaining_volume"]])
self.data["min_basin_volume"] = pl_to_dict_with_tuple(
            self.index["state"].select("BS", c("volume").struct.field("min")))
self.data["max_basin_volume"] = pl_to_dict_with_tuple(
    self.index["state"].select("BS", c("volume").struct.field("max")))
self.data["powered_volume_enabled"] = {None: self.powered_volume_enabled}

self.data["discharge_volume"] = pl_to_dict_with_tuple(self.discharge_volume.filter(c("sim_nb") == self.sim_nb)[["TB", "discharge_volume"]])  
self.data["market_price"] = pl_to_dict(self.market_price.filter(c("sim_nb") == self.sim_nb)[["T", "avg"]])
if not self.global_price:
    self.data["neg_unpowered_price"] = {
        None: self.market_price.filter(c("sim_nb") == self.sim_nb)["avg"].quantile(0.5 + self.quantile)}
    self.data["pos_unpowered_price"] = {
        None: self.market_price.filter(c("sim_nb") == self.sim_nb)["avg"].quantile(0.5 - self.quantile)}

self.data["powered_volume"] = pl_to_dict(self.powered_volume.filter(c("sim_nb") == self.sim_nb)[["H", "powered_volume"]])
self.data["volume_buffer"] = pl_to_dict(self.volume_buffer.filter(c("sim_nb") == self.sim_nb)[["H", "volume_buffer"]])

self.data["min_flow"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "flow"]])  
self.data["min_power"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "electrical_power"]])  
self.data["d_flow"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "d_flow"]])  
self.data["d_power"] = pl_to_dict_with_tuple(hydropower_state[["HQS", "d_electrical_power"]])

In [ ]:
second_stage.optimization_results

{'start_basin_volume': shape: (2, 3)
 ┌─────┬────────┬────────────────────┐
 │ B   ┆ sim_nb ┆ start_basin_volume │
 │ --- ┆ ---    ┆ ---                │
 │ u32 ┆ i32    ┆ f64                │
 ╞═════╪════════╪════════════════════╡
 │ 0   ┆ 0      ┆ 14.8               │
 │ 1   ┆ 0      ┆ 0.0                │
 └─────┴────────┴────────────────────┘,
 'remaining_volume': shape: (2, 3)
 ┌─────┬────────┬──────────────────┐
 │ H   ┆ sim_nb ┆ remaining_volume │
 │ --- ┆ ---    ┆ ---              │
 │ u32 ┆ i32    ┆ i32              │
 ╞═════╪════════╪══════════════════╡
 │ 0   ┆ 0      ┆ 0                │
 │ 1   ┆ 0      ┆ 0                │
 └─────┴────────┴──────────────────┘,
 'flow': shape: (0, 0)
 ┌┐
 ╞╡
 └┘,
 'power': shape: (0, 0)
 ┌┐
 ╞╡
 └┘,
 'spilled_volume': shape: (0, 0)
 ┌┐
 ╞╡
 └┘,
 'basin_volume': shape: (0, 0)
 ┌┐
 ╞╡
 └┘}

In [ ]:
self.first_stage_results

NameError: name 'self' is not defined

In [ ]:
self = second_stage

divisors: int = int(self.timestep / self.first_stage_timestep)

offset = divisors - self.first_stage_results.height%divisors
self.powered_volume = self.first_stage_results.select(
        c("T"), 
        cs.starts_with("powered_volume").name.map(lambda c: c.replace("powered_volume_", "")),
    ).group_by(((c("T") + offset)//divisors).alias("sim_nb"), maintain_order=True)\
    .agg(pl.all().exclude("sim_nb", "T").sum())\
    .unpivot(
        index="sim_nb", variable_name="H", value_name="powered_volume"
    ).with_columns(
        c("H").cast(pl.UInt32).alias("H")
    )

In [ ]:
self.index["hydro_power_plant"]\
            .select(
                c("H").cast(pl.Utf8), 
                c("rated_flow") * self.real_timestep.total_seconds() * self.volume_factor * self.buffer
            )

H,rated_flow
str,f64
"""0""",0.002016
"""1""",null


In [ ]:
self.process_timeseries()

In [ ]:
self.discharge_volume 

T,timestamp,discharge_volume,max_discharge_volume,min_discharge_volume,sim_nb,B,TB
u32,"datetime[μs, UTC]",f64,f64,f64,u32,i32,list[i64]
1,2020-01-01 00:00:00 UTC,0.0006,0.0006,0.0006,0,0,"[1, 0]"
2,2020-01-01 01:00:00 UTC,0.0005985,0.0005985,0.0005985,0,0,"[2, 0]"
3,2020-01-01 02:00:00 UTC,0.000597,0.000597,0.000597,0,0,"[3, 0]"
4,2020-01-01 03:00:00 UTC,0.000596,0.000596,0.000596,0,0,"[4, 0]"
5,2020-01-01 04:00:00 UTC,0.000595,0.000595,0.000595,0,0,"[5, 0]"
…,…,…,…,…,…,…,…
92,2020-12-31 19:00:00 UTC,0.000404,0.000404,0.000404,91,0,"[92, 0]"
93,2020-12-31 20:00:00 UTC,0.000404,0.000404,0.000404,91,0,"[93, 0]"
94,2020-12-31 21:00:00 UTC,0.000404,0.000404,0.000404,91,0,"[94, 0]"


In [ ]:
plot_first_stage_result(
    simulation_results=first_stage.optimization_results, 
    time_divider=7
).show()

AttributeError: 'NoneType' object has no attribute 'show'

In [ ]:
first_stage.water_flow_factor

BH,water_factor
list[i64],i64
"[0, 0]",-1
"[0, 1]",1
"[1, 0]",1
"[1, 1]",-1


In [ ]:
first_stage.optimization_results

T,timestamp,market_price,max_market_price,min_market_price,basin_volume_0,basin_volume_1,powered_volume_0,powered_volume_1,hydro_power_0,hydro_power_1,income
u32,"datetime[μs, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2020-01-01 00:00:00 UTC,31.815417,41.14,26.92,0.811261,0.0,-2.5211e-8,-0.0,-0.000001,0.0,-0.000019
1,2020-01-02 00:00:00 UTC,39.168333,46.29,29.03,0.81204,0.0,0.160475,-0.0,6.319766,0.0,150.852457
2,2020-01-03 00:00:00 UTC,40.166667,44.33,31.28,0.804,0.160474,0.160475,-0.0,6.319766,0.0,154.697426
3,2020-01-04 00:00:00 UTC,35.39875,43.51,20.45,0.795941,0.320949,0.119503,-0.0,4.706234,0.0,101.526101
4,2020-01-05 00:00:00 UTC,37.708333,46.43,30.54,0.790114,0.440452,0.160475,-0.0,6.319766,0.0,145.22943
…,…,…,…,…,…,…,…,…,…,…,…
361,2020-12-27 00:00:00 UTC,23.205,44.59,4.06,0.826491,0.152979,0.0,-0.152979,0.0,-8.493146,-85.196811
362,2020-12-28 00:00:00 UTC,43.325417,57.64,14.25,0.835451,0.0,0.0,-0.0,0.0,0.0,0.0
363,2020-12-29 00:00:00 UTC,48.305833,57.12,32.47,0.836015,0.0,0.160475,-0.0,6.319766,0.0,186.044516


In [ ]:
extract_optimization_results(
            model_instance=first_stage.model_instance, var_name="spilled_volume"
        )


T,B,spilled_volume
u32,u32,f64
0,0,0.0
0,1,0.0
1,0,0.0
1,1,0.0
2,0,0.0
…,…,…
363,1,0.160475
364,0,0.0
364,1,0.160475


In [ ]:

turbined_power = extract_optimization_results(
        model_instance=model_instance, var_name="turbined_power"
    )

turbined_power = pivot_result_table(
    df = turbined_power, on="H", index=["T"],
    values="turbined_power")

simulation_results: pl.DataFrame = market_price\
    .join(basin_volume, on = "T", how="inner")\
    .join(turbined_volume, on = "T", how="inner")\
    .join(pumped_volume, on = "T", how="inner")\
    .join(pumped_power, on = "T", how="inner")\
    .join(turbined_power, on = "T", how="inner")\
    .with_columns(
        ((
            pl.sum_horizontal(cs.starts_with("turbined_power")) -
            pl.sum_horizontal(cs.starts_with("pumped_power"))
        ) * c("T").replace_strict(nb_hours_mapping, default=None) * c("market_price")).alias("income")
    )
    
hydro_name = list(map(str, list(model_instance.H))) # type: ignore

simulation_results = simulation_results.with_columns(
    pl.struct(cs.ends_with(hydro) & ~cs.starts_with("basin_volume"))
    .pipe(remove_suffix).alias("hydro_" + hydro) 
    for hydro in hydro_name
).select(    
    ~(cs.starts_with("turbined") | cs.starts_with("pumped")) # type: ignore
)


NameError: name 'model_instance' is not defined